In [ ]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Parametros
precio_casa = 465000
ahorros = 125000
prestamo = 372000
itp = 0.06 # Impuesto transmisiones patrimoniales (Madrid)
notario = 1600
registro = 800
gestoria = 350
tasacion = 875
anios = 30
tin = 0.013 # Tasa interes nominal (es anual no mensual)

In [ ]:
# # Calculos derivados
# def calc_prestamo_inicial(precio_casa, ahorros, default_inicial = None):
#     if not default_inicial:
#         prestamo = precio_casa - (ahorros - itp*precio_casa - notario - registro - gestoria - tasacion)
#     else:
#         prestamo = default_inicial
#     return prestamo

# def calc_serie_cuota_mensual(tin, anios, precio_casa, ahorros, as_array = True, default_inicial = None):
#     i = tin / 12
#     r = 1/(1+i)
#     meses = 12 * anios
#     prestamo = calc_prestamo_inicial(precio_casa, ahorros,default_inicial)
#     cuota = (1-r)*prestamo / (r - r**(meses+1)) # No uso aproximacion por exponencial
#     if as_array:
#         return cuota * np.ones(meses)
#     else:
#         return cuota

# def calc_series_mensuales_deudas_intereses_principal(tin, anios, prestamo,cuota_mensual,extra_amortizaciones=0):
#     i = tin/12
#     meses = 12 * anios
#     # Initialization
#     deuda_mensual_prepago = np.zeros(meses)
#     deuda_mensual_postpago = np.zeros(meses)
#     interes_mensual = np.zeros(meses)
#     principal_mensual = np.zeros(meses)
#     # Fisrt values
#     interes_mensual[0] = i * prestamo
#     deuda_mensual_prepago[0] = prestamo + interes_mensual[0] # El primer mes es el total mas el interes
#     deuda_mensual_postpago[0] = deuda_mensual_prepago[0] - cuota_mensual[0]
#     principal_mensual[0] = cuota_mensual[0] - interes_mensual[0]
    
#     for mes in range(1,meses):
#         interes_mensual[mes] = deuda_mensual_prepago[mes-1]*i
#         deuda_mensual_prepago[mes] = (deuda_mensual_prepago[mes-1] - cuota_mensual[mes-1]) + interes_mensual[mes]
#         deuda_mensual_postpago[mes] = deuda_mensual_prepago[mes] - cuota_mensual[mes]
#         principal_mensual[mes] = cuota_mensual[mes] - interes_mensual[mes]
        
#     return pd.DataFrame(data={"interes_mensual":interes_mensual, "deuda_mensual_prepago":deuda_mensual_prepago,"deuda_mensual_postpago":deuda_mensual_postpago, "principal_mensual":principal_mensual}, index=np.arange(1,meses+1))


In [ ]:
def calc_serie_cuota_mensual(tin, anios, prestamo, as_array = True):
    i = tin / 12
    r = 1/(1+i)
    meses = 12 * anios
    cuota = (1-r)*prestamo / (r - r**(meses+1)) # = i * prestamo / (1 - (1+i)^(-meses))
    if as_array:
        return cuota * np.ones(meses)
    else:
        return cuota

def calc_series_mensuales_deudas_intereses_principal(tin, anios, prestamo,cuota_mensual,extra_amortizaciones=0):
    i = tin/12
    meses = 12 * anios
    # Initialization
    capital_vivo = np.zeros(meses+1)
    capital_amortizado = np.zeros(meses+1)
    intereses = np.zeros(meses+1)
    amortizacion = np.zeros(meses+1)
    # Mes 0 - Valores
    intereses[0] = 0
    amortizacion[0] = 0
    capital_vivo[0] = prestamo
    capital_amortizado[0] = 0

    for mes in range(1,meses+1):
        intereses[mes] = capital_vivo[mes-1]*i # Pagas el i sobre lo que falta por pagar (capital vivo)
        amortizacion[mes] = cuota_mensual[mes-1] - intereses[mes] # Cuota es intereses + amortizacion (cuto es en mes -1 porque son "#meses" cuotas no "#meses+1" como el resto que tienen el mes 0)
        capital_vivo[mes] = capital_vivo[mes-1] - amortizacion[mes]
        capital_amortizado[mes] = capital_amortizado[mes-1] + amortizacion[mes]
        
    return pd.DataFrame(data={"intereses":intereses, "amortizaciones":amortizacion,"capital_vivo":capital_vivo, "capital_amortizado":capital_amortizado}, index=np.arange(meses+1))

def calc_tae(prestamo, cuotas, gastos):
    flujos_caja = np.zeros(len(cuotas)+1) # +1 por el Mes 0
    flujos_caja[0] = -prestamo
    flujos_caja[1:] = cuotas + gastos
    TIR = npf.irr(flujos_caja)
    TAE = (1 + TIR)**12 - 1
    return {"TAE":TAE,"TIR":TIR}

In [ ]:
cuotas = calc_serie_cuota_mensual(tin=tin, anios=anios, prestamo=prestamo, as_array=True)
df = calc_series_mensuales_deudas_intereses_principal(tin, anios,prestamo,cuotas)

# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"intereses"].iloc[1:],
                    mode='lines',
                    name='Interes pagado mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"amortizaciones"].iloc[1:],
                    mode='lines',
                    name='Amortizacion mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=cuotas,
                    mode='lines',
                    name='Cuota mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero de mes")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Intereses vs principal con tin {100*tin}%"
)

fig.show()

In [ ]:
# TIN vs Total pagado
tin_a = np.arange(0.008,0.0185,0.0005)
total_pagado = np.zeros(len(tin_a))
for tin_pos, tin in enumerate(tin_a):
    total_pagado[tin_pos] = sum(calc_serie_cuota_mensual(tin=tin, anios=anios, prestamo=prestamo, as_array=True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=100*tin_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Valor del TIN fijo en %")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text="TIN vs Total pagado con 30 anios"
)

fig.show()

In [ ]:
# Anios vs Total pagado
anios_a = np.arange(15,30,1)
total_pagado = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    total_pagado[anios_pos] = sum(calc_serie_cuota_mensual(0.013, anios, precio_casa, ahorros, as_array = True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text="Anios vs Total pagado con tin 1.29"
)

fig.show()

In [ ]:
# Anios vs Cuota mensual
anios_a = np.arange(15,30,1)
cuota_mensual = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    cuota_mensual[anios_pos] = calc_serie_cuota_mensual(0.013, anios, precio_casa, ahorros, as_array = False)

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=cuota_mensual,
                    mode='lines',
                    name='Cuota Mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text="Cuota Mensual vs Anios con tin 1.29"
)

fig.show()